# 2. Recurrent Neural Networks basics

**Objective**

- Understanding RNN capabilities with toy problems;
- Getting familiar with `torch` RNN tools.

**Resources and references**
- [Stanford cheat-sheet on RNNs](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks#overview)
- Andrej Karpathy, ["The Unreasonable Effectiveness of Recurrent Neural Networks"](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- J. Brownlee, ["5 Examples of Simple Sequence Prediction Problems for LSTMs"](https://machinelearningmastery.com/sequence-prediction-problems-learning-lstm-recurrent-neural-networks/)
- [Christopher Olah's blog about LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) (all his blog posts are great by the way)
- [A very nice experimentation to play with LSTM](https://distill.pub/2016/handwriting/) (to go further)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

## Summary

- **1. Working with RNNs in PyTorch**
  - *Elman's RNN*
  - *Running an RNN*
  - *What's happening inside?*
- **2. Task typology**
    - *Overview*
    - *2.1. Toy problem: Many-to-one*
    - *2.2. Toy problem: One-to-many*
- **3. Project: handwriting synthesis**

## 1. Working with RNNs

RNNs are a little bit different from their classical feedfoward cousins (you know their most common flavor: multilayerd perceptrons.)

In a feedforward networks, fome function $f_{w,b}$ is applied on inputs $x$ to compute output $y$:

<p align="center">
  <img src="./static/perceptron.png" />
</p>

$$
y = f_{w,b}(x) = act(\sum_i w_i x_i + b)
$$

$w$ (and $b$) are parameters of the network, and represents the weights of the connections between the neurons and their inputs (and a constant bias parameter). $act$ is the activation function.

RNN are just slightly different: they take as input not only $x$, the current input, but also some of their previous computations $h$. In the simplest case, $h$ is equivalent to the output $y$ of our perceptron. We call it **the internal or hidden state**.

$$
y_t = h_t = act(\sum_i w^{in}_i x_{t, i} + \sum_j w^h h_{t-1, j} + b)
$$

<p align="center">
  <img src="./static/rnn.png" />
</p>


Note how we added some indices $t$ everywhere: now **time matters**. Your data points $x$ are no longer independent from each other, they form **sequences**, as we have seen in previous chapter.

As simple RNNs are really just perceptrons with a supplementary input playing the role of a memory tape, we can also represent an RNN as a repeated application of a perceptron with two different inputs ($x_t$ and $h_{t-1}$):

<p align="center">
  <img src="./static/rnn-unfold.png" />
</p>

We will call this kind of representation **unfolded representations**, as recurrence and time are projected on the horizontal axis.

The perceptron is here first applied at time $t$ on $x_t$ and $h_{t-1}$, its previous internal state. We have defined two different sets of weights $w_{in}$ and $w_h$ to parameterize the neuron, but this does not change much to the equation. Once the new state $h_{t}$ is computed, we move on to the next input $x_{t+1}$, and so on.

#### Elman's RNN

The neuron we have just defined is the building block of **Elman networks**, (from Jeffrey Locke Elman, their inventor).
Elman's network, also called "simple RNNs" or just "RNNs", usually define an additional set of output weights to compute $y_t$ from $h_t$. This is like adding a layer of connections on top the internal state $h_t$ to perform the task we want (prediction, classification...)

Simple RNNs are available in Pytorch through the [`RNN`](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) class. They do not implement the any output layer of weights. It will be up to you to add some more computations on top of some $h_t$ to perform your task. For now, let's look at how those neural networks work wihtout performing any task.

In [ ]:
# Create an 100-neurons RNN dealing with one dimensional inputs
rnn = ...

In [ ]:
print("Wh weights shape:", rnn.weight_hh_l0.shape)
print("Win weights shape:", rnn.weight_ih_l0.shape)

#### Running an RNN

Let's try running the RNN on some dummy data, here two succesive sine waves with different frequencies.

In [ ]:
wave1 = np.sin(np.linspace(0, 4*np.pi, 100, endpoint=False))  # just some sinusoidal waves
wave2 = np.sin(np.linspace(4*np.pi, 8*np.pi, 100)*5)

wave = np.concatenate([wave1, wave2])

plt.plot(wave)
plt.show()

In [ ]:
# Always convert arrays to torch.Tensor!
x = torch.Tensor(wave)

# RNNs expect inputs of shape (sequence length, input dimension) (or (batch_size, sequence_length, input_dimension))
# so here we need to reshape x from (100,) to (100, 1) (or (1, 100, 1))
x =  ...

In [ ]:
with torch.no_grad():  # deactivate automatic gradient computation, we don't need it here
    # Apply the RNN on the wave. What are the outputs representing?
    all_outputs, last_output = ...

print(all_outputs.size(), "\n", last_output.size())
print(all_outputs, "\n", last_output)

#### What's happening inside?

Our 100-dimensional vectors represent the hidden states of our 100 neurons, for all input data steps.
Let's have a look at those hidden states.

In [ ]:
# Plot the activity of some random neurons inside the RNN
...
# The activity of all neurons inside the RNN at a timestep t is called the "hidden_state".

## Task typology

### Overview: What do we do with it ?

There are many ways of using RNNs in a neural network architecture, as they allow to fold and unfold time in your data.


For instance, you can imagine generating long captions for single images, going from a static component (an image) to a sequence of words of any length. On the contrary, a sequence of words could be compressed into a single label representing overall sentiment of a sentence ("happy" or "sad"). Or, you can use RNNs to translate a French sentence of size $n$ into an English sentence of size $m$. 

As you can see, RNNs allow to play along the time axis in various manners, depending on the task at hand, which make them incredibly more powerful than feedforwad neural networks in many situations.

In the following figures, green vectors $x$ represent input sequence or vector, green vector $a$ represents hidden states, and red vectors $y$ represent output of the neural network, which can be equal to $a$ or post-processed by some other neural network, like a layer of neurons trained to output class probabilities. RNNs are represented unfolded in time to make sequences apparent.

These variations of RNN behavior fall into four main classes of networks.

- **Many-to-many models**: from a sequence/timeseries, output another sequence/timeseries of **same length.** Ex: online speech recognition.
<p align="center">
  <img src="./static/rnn-many-to-many-same-ltr.png" width=500/>
</p>

- **Many-to-one, or seq-to-vec models**: from a sequence, output a single vector. Ex: sequence classification tasks, like sentence sentiment analysis.
<p align="center">
  <img src="./static/rnn-many-to-one-ltr.png" width=500/>
</p>

- **One-to-many, or vec-to-seq models**: from a single vector, output a sequence. Ex: image captionning or music generation.
<p align="center">
  <img src="./static/rnn-one-to-many-ltr.png" width=500/>
</p>

- **Encoder-decoder models**: from a sequence/timeseries, output another sequence/timeseries of **different length.** Ex: machine translation.
<p align="center">
  <img src="./static/rnn-many-to-many-different-ltr.png" width=500/>
</p>


### Many-to-one: the pocket calculator network

Let's design a neural network able to perform a **simple classfication task**: *tell if an arithmetical expression will give negative or positive results.*

Inputs are sequences of numbers as follow:

- every step of input is made of two number: an operand and an operator.
- the first number (the operand) is the number on which we will perform the operation
- the second number (the operator) is either 1 or 0. 1 indicates that we want to add that number to the total, 0 indicates that we want to substract that number.

Hence, the following expression:
$$+ 1 + 2 - 5 + 4 - 6$$

is represented by the input sequence:

```
[[1, 1], 
 [2, 1], 
 [5, 0], 
 [4, 1],
 [6, 0]]
```

The target class is the expected sign of the total (here  -4, so **negative class**)

All sequences are 5 operands long. All operands are single digits (0 to 9).

In [ ]:
def make_operand_sequences(n_sequences, seq_length=5):
    digits = list(range(0, 10))
    operators = [0, 1]
    op_symbols = ["-", "+"]
    x = np.zeros((n_sequences, seq_length, 2))
    y = np.zeros(n_sequences)
    for i in range(n_sequences):
        operands = np.random.choice(digits, seq_length, replace=True)
        ops = np.random.choice(operators, seq_length, replace=True)
        seq = np.vstack([operands, ops])
        x[i] = seq.T
        expr = ""
        for digit, op in zip(operands, ops):
            expr += op_symbols[operators.index(op)] + str(digit)
        total = eval(expr)
        y[i] = 1 if total > 0 else 0
    return x, y

In [ ]:
x, y = make_operand_sequences(1000)

In [ ]:
class Calculator(torch.nn.Module):

    def __init__(self, hidden_size):
        super().__init__()
        
        # Remember you can store and do everything 
        # you need in nn.Module objects
        # by defining your own methods and attributes.
        # Pytorch is very permissive!
        
        # Number of neurons in the RNN
        self.hidden_size = hidden_size
        
        # Create an RNN for the task
        # and do not forget batch_first=True
        # and an output classification layer
        self.rnn = ...
        self.fc = ...

        # Note that we won't add an activation function like softmax to the last layer
        # for now. Pytorch prefers training classifier using unormalized
        # outputs when choosing cross-entropy loss (see documentation).

    def forward(self, x):
        # Perform forward (prediction) pass.
        # Get only the last state on a sequence and pass it to the classification layer.
        # This state will represent the accumulated information
        # along the sequence of inputs.
        
        ...
        
        output = ...
        
        return output

In [ ]:
device = "cpu"  # change it to "cuda" if you can run on GPU

network = Calculator(hidden_size=100).to(device)

print(network)

#### Training loop ####
num_epochs = 50  # change it as you wish

network.train()  # training mode: on

# Loss: cross entropy (this is a classfication task!)
criterion = torch.nn.CrossEntropyLoss()

# Learning rule (optimizer). We will go with a refinement
# of the gradient descent, called Adam.
opt = torch.optim.Adam(network.parameters(), lr=1e-3)


# Convert everything to torch.Tensor and create a Dataset and 
# a DataLoader objects.
# They are used to hold your data during training, and help
# you define batch size or data transformations for instance.
# For now, we need only a simple TensorDataset (taking Tensors as 
# inputs) and a DataLoader with our chosen batch size.
batch_size = 10  # change it as you wish

x_train = torch.Tensor(x).to(device)
y_train = torch.Tensor(y).to(torch.long).to(device)  # class labels must be converted to torch.long

# Create TensorDataset and DataLoader
dataset = torch.utils.data.TensorDataset(x_train, y_train)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

losses = []
for e in range(num_epochs):

    epoch_loss = []

    for x_i, y_i in dataloader:  # for each batch of data
        
        # training loop !
        ...
        
        loss = ...

        # Save loss values somewhere to plot the learning curve
        loss = loss.detach().numpy()
        epoch_loss.append(loss)
        
        print(f"Epoch {e} - Loss {np.mean(epoch_loss):.3f}", end="\r")

    losses.append(np.mean(epoch_loss))

In [ ]:
plt.plot(losses)

In [ ]:
x_test, y_test = make_operand_sequences(100)

In [ ]:
# Test network
network.eval()
with torch.no_grad():
    y_pred = network(torch.Tensor(x_test))

In [ ]:
print("Accuracy:", np.sum(np.argmax(y_pred.numpy(), axis=1) == y_test) / 100)

### One-to-many: the oscillator

One-to-many networks require a bit of mental exercise to set up a task properly. In the following example, we will try to solve a rather simple problem: can we train an RNN to generate a sinusoidal wave (many outputs) given a frequency as (single) input?

To generate a sequence from a vector, an RNN needs two conditions (which will recall the basic recurrence reasoning in maths; hence the name of RNN):
- a good initial condition;
- a recurrence relationship between RNN's outputs.

<p align="center">
  <img src="./static/rnn-one-to-many-ltr.png" width=500/>
</p>

The initial condition here is the green $x$ in the figure, or the "one" in "one-to-many". It is here to drive the network in an initial state from which it can autonomously generate the whole sequence. In our case, this could be a simple one-hot vector representing the frequency of the sinusoidal wave we want to generate. 

This $x$ can be fed to the network as a first input before the network receives its own previous outputs as input. This is exactly what can be seen in the schema. This is called a pre-inject.

Other techniques exist. $x$ can also be used directly as a first hidden state for the RNN. This is called an init-inject. Or $x$ can be added to the network outputs at every timestep, like a constant reminder of the initial goal. This is called a par-inject.

Once the initialization is done, the network will then work on its own outputs to generate the whole sequence, by recurrence.

In this example, we will do an init-inject initialization. We will project the 3-d vector holding frequency information into hidden state space, and use the result to initialize the first hidden state of the network (which is initialize to 0 by default). This can be done by adding a layer of connections between the frequency vector and the hidden space, that will "reshape" the frequency vector to be as big as the hidden state vector. Once reshaped, the transformed frequency can be directly fed as hidden state input to the RNN.

From there, the network will have to predict a first point of the wave. This point will serve as next input to the network, and so on.


In [ ]:
def make_waves(n_sequences, frequencies, length):
    x = np.zeros((n_sequences, 1, len(frequencies)))
    y = np.zeros((n_sequences, length, 1))
    for i in range(n_sequences):
        fs = np.random.choice(frequencies)
        waves = np.sin(2*np.pi*fs*np.linspace(0, 1, length))
        fs_idx = frequencies.index(fs)
        x[i, 0, fs_idx] = 1  # one hot encode frequencies: 0 for all, 1 for the current frequency
        y[i, :, 0] = waves
    
    # return shape (n, length, 1)
    return x, y

In [ ]:
frequencies = [1, 5, 10]  # frequency values
length = 50  # length of the sinusoidal wave

x, y = make_waves(1000, frequencies, length)

In [ ]:
# Shape: [n, length, dimension]
# Note that length is 1 here, as we only give a single input to the network.
# One x per wave, representing the frequency class
print(x.shape, "\n", x)

In [ ]:
class SineWaveGenerator(torch.nn.Module):

    def __init__(self, hidden_size, n_frequencies, length):
        super().__init__()

        self.hidden_size = hidden_size
        self.length = length

        # Create a layer to transform frequency into hidden states
        self.input_to_hidden = ...
        # Create an RNN
        self.rnn = ...
        # Create an output layer to transform hidden states into a 1-d point of sinusoidal wave
        self.hidden_to_output = ...

    def forward(self, freq):
                
        # store outputs
        outputs = torch.zeros(x.size(0), self.length, 1).to(freq.device)
        
        # Mind the shapes here:
        # RNN outputs do not support batch_first=True... 
        # You may need to transpose things to keep everything coherent.

        # init-inject
        # You should go from input vector of shape [n, 1, frequencies]
        # to init hidden state of shape [1, n, hidden_size]
        
        hidden = ...
        
        # first input is null
        # zeros: [n, length=1, dim]
        x = ...
        
        # Generate the wave by recurrence
        for i in range(0, self.length):
                
            y = ...
            
            # store outputs
            outputs[:, i, 0] = y.squeeze()
            
            # Use output as next input
            x = y
            
        return outputs

In [ ]:
network = SineWaveGenerator(hidden_size=100, n_frequencies=3, length=length)

print(network)

# Training loop
num_epochs = 10
batch_size = 10

losses = []

# Ok now do it yourselves

In [ ]:
plt.plot(losses)

In [ ]:
x_test, y_test = make_waves(100, frequencies, length)

In [ ]:
network.eval()
with torch.no_grad():
    y_pred = network(torch.Tensor(x_test))

In [ ]:
print("MSE:", np.mean((y_test - y_pred.numpy())**2))